In [3]:
import pandas as pd
import numpy as np 

In [4]:
df = pd.read_csv(r"C:\Users\cmoli\OneDrive\Documentos\GitHub\Titanic\data\titanic_limpio.csv")

In [5]:
df.head(2) #leer

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Tiene_cabina
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,sin datos,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1


### Porcentaje de supervivientes

In [6]:
porcentaje_supervivientes = (df['Survived'].mean()) * 100
porcentaje_supervivientes

np.float64(38.38383838383838)

In [7]:
tabla_clase = df.groupby("Pclass")["Survived"].agg(["count", "mean","sum" ]) #agrupamos por clase y calculamos la media de supervivencia
tabla_clase.columns = ["Conteo", "Media", "Supervivientes"] #cambiamos el nombre de las columnas
tabla_clase = tabla_clase.sort_index() #ordenamos por índice
tabla_clase #mostramos la tabla

,Conteo,Media,Supervivientes
Pclass,,,
1,216,0.629630,136
2,184,0.472826,87
3,491,0.242363,119


### Apellido más repetidos por clase

In [8]:
df['Apellido'] = df['Name'].str.split(',').str[0]

apellidos_por_clase = df.groupby(['Pclass', 'Apellido']).size().reset_index(name='Frecuencia') # contamos con el size con el nombre frecuencia basado en pcclass y apellido

# apellidos más comunes por clase
apellidos_mas_comunes = apellidos_por_clase.sort_values(['Pclass', 'Frecuencia'], ascending=[True, False])

# creamos un DataFrame con los 3 apellidos más comunes por clase
top_apellidos_por_clase = (apellidos_mas_comunes.groupby('Pclass')
                           .apply(lambda x: x.nlargest(3, 'Frecuencia'))
                           .reset_index(drop=True))


top_apellidos_por_clase



C:\Users\cmoli\AppData\Local\Temp\ipykernel_8348\3160238038.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.nlargest(3, 'Frecuencia'))


,Pclass,Apellido,Frecuencia
0,1,Carter,4
1,1,Fortune,4
2,1,Allison,3
3,2,Brown,3
4,2,Collyer,3
5,2,Hart,3
6,3,Andersson,9
7,3,Sage,7
8,3,Goodwin,6


In [12]:
apellido_mas_comun = df['Apellido'].value_counts().nlargest(1)
apellido_mas_comun

Apellido
Andersson    9
Name: count, dtype: int64

### Precio medio por clase

In [13]:
precio_medio_por_clase = df.groupby('Pclass')['Fare'].agg(['mean', 'median', 'min', 'max', 'std'])
precio_medio_por_clase = precio_medio_por_clase.round(2)
precio_medio_por_clase

,mean,median,min,max,std
Pclass,,,,,
1,84.15,60.29,0.0,512.33,78.38
2,20.66,14.25,0.0,73.50,13.42
3,13.68,8.05,0.0,69.55,11.78


### Edad media por clase

In [14]:
edad_por_clase = df.groupby('Pclass')['Age'].agg(['mean', 'median', 'min', 'max', 'std', 'count'])
edad_por_clase = edad_por_clase.round(2)
edad_por_clase

,mean,median,min,max,std,count
Pclass,,,,,,
1,36.81,35.0,0.92,80.0,14.18,216
2,29.77,28.0,0.67,70.0,13.58,184
3,25.93,28.0,0.42,74.0,10.70,491


### Supervivientes por rangos de edad

In [18]:
bins = [0, 5, 12, 18, 35, 60, np.inf] #usando bins para establecer rangos
labels = ['Bebé (0-5)', 'Niño (5-12)', 'Adolescente (12-18)', 'Joven (18-35)', 'Adulto (35-60)', 'Mayor (60+)']
df['Rango_Edad'] = pd.cut(df['Age'], bins=bins, labels=labels)# creamos nuueva col para esos rangos

In [19]:
supervivientes_por_edad = df.groupby('Rango_Edad')['Survived'].agg(['count', 'sum', 'mean'])
supervivientes_por_edad.columns = ['Total', 'Supervivientes', 'Porcentaje']
supervivientes_por_edad['Porcentaje'] = supervivientes_por_edad['Porcentaje'] * 100

C:\Users\cmoli\AppData\Local\Temp\ipykernel_8348\4062204627.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  supervivientes_por_edad = df.groupby('Rango_Edad')['Survived'].agg(['count', 'sum', 'mean'])


In [20]:
supervivientes_por_edad

,Total,Supervivientes,Porcentaje
Rango_Edad,,,
Bebé (0-5),44,31,70.454545
Niño (5-12),25,9,36.000000
Adolescente (12-18),70,30,42.857143
Joven (18-35),535,189,35.327103
Adulto (35-60),195,78,40.000000
Mayor (60+),22,5,22.727273


### % de supervencia entre hombres, mujeres y ninos por clase

In [21]:
df['GrupoEdadSexo'] = 'Adulto'
df.loc[df['Age'] < 12, 'GrupoEdadSexo'] = 'Niño'

df.loc[(df['Age'] >= 12) & (df['Sex'] == 'male'), 'GrupoEdadSexo'] = 'Hombre'
df.loc[(df['Age'] >= 12) & (df['Sex'] == 'female'), 'GrupoEdadSexo'] = 'Mujer'

supervivencia_grupo_clase = pd.pivot_table(
    df, 
    values='Survived',
    index='Pclass',
    columns='GrupoEdadSexo',
    aggfunc='mean'
) * 100

supervivencia_grupo_clase.round(2)

GrupoEdadSexo,Hombre,Mujer,Niño
Pclass,,,
1,35.29,97.85,75.00
2,8.08,91.18,100.00
3,12.07,50.41,40.43


In [23]:
conteo_grupo_clase = pd.pivot_table(
    df, 
    values='PassengerId',
    index='Pclass',
    columns='GrupoEdadSexo',
    aggfunc='count'
)


conteo_grupo_clase

GrupoEdadSexo,Hombre,Mujer,Niño
Pclass,,,
1,119,93,4
2,99,68,17
3,323,121,47


In [24]:
# tabla combinada con porcentaje y conteo
print("\nTabla combinada (porcentaje % [conteo]):")
tabla_combinada = supervivencia_grupo_clase.round(1).astype(str) + '% [' + conteo_grupo_clase.astype(str) + ']'
tabla_combinada


Tabla combinada (porcentaje % [conteo]):


GrupoEdadSexo,Hombre,Mujer,Niño
Pclass,,,
1,35.3% [119],97.8% [93],75.0% [4]
2,8.1% [99],91.2% [68],100.0% [17]
3,12.1% [323],50.4% [121],40.4% [47]


### Supervivientes por sexo

In [25]:
supervivientes_por_sexo = df.groupby('Sex')['Survived'].agg(['count', 'sum', 'mean'])
supervivientes_por_sexo.columns = ['Total', 'Supervivientes', 'Porcentaje']
supervivientes_por_sexo['Porcentaje'] = supervivientes_por_sexo['Porcentaje'] * 100

supervivientes_por_sexo.round(2)

,Total,Supervivientes,Porcentaje
Sex,,,
female,314,233,74.20
male,577,109,18.89
